In [1]:
#Import dependencies
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException, TimeoutException
import time
import requests
from bs4 import BeautifulSoup
import re
import config
import json
import os
from urllib.parse import urlparse
from selenium.webdriver.chrome.service import Service

In [2]:

#specify the path to chromedriver.exe (download and save on your computer)
serv = Service(executable_path=r'C:\Users\kirti\chromedriver-win64\chromedriver-win64\chromedriver.exe')
driver = webdriver.Chrome(service=serv)

#open the webpage
driver.get("https://www.instagram.com/")

In [3]:
#target username
username = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='username']")))
password = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input[name='password']")))

#enter username and password
username.clear()
username.send_keys('your_username')
password.clear()
password.send_keys('your_password')

#target the login button and click it
button = WebDriverWait(driver, 2).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "button[type='submit']"))).click()

In [4]:
# not_button = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div/div/div/div')))

# not_button.click()

In [5]:
# not_button2 = WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, '/html/body/div[3]/div[1]/div/div[2]/div/div/div/div/div[2]/div/div/div[3]/button[2]')))
# not_button2.click()

In [6]:
# Wait up to 10 seconds for the search button to be clickable on the web page
reel_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, 'svg[aria-label="Explore"]')))

# Click the search button once it becomes clickable
reel_button.click()

In [7]:
# Get the initial page height
initial_height = driver.execute_script("return document.body.scrollHeight")

# Create a list to store htmls
soups = []
count = 0
while True:
    # Scroll down to the bottom of the page
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

    # Wait for a moment to allow new content to load (adjust as needed)
    time.sleep(10)
    
    # Parse the HTML
    html = driver.page_source
    
    # Create a BeautifulSoup object from the scraped HTML
    soups.append(BeautifulSoup(html, 'html.parser'))

    # Get the current page height
    current_height = driver.execute_script("return document.body.scrollHeight")

    if count == 5:
        break  # Exit the loop when you can't scroll further

    initial_height = current_height  # Update the initial height for the next iteration
    count+=1

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=123.0.6312.123)
Stacktrace:
	GetHandleVerifier [0x00007FF66E527032+63090]
	(No symbol) [0x00007FF66E492C82]
	(No symbol) [0x00007FF66E32EC65]
	(No symbol) [0x00007FF66E30CA7C]
	(No symbol) [0x00007FF66E39D687]
	(No symbol) [0x00007FF66E3B2AC1]
	(No symbol) [0x00007FF66E396D83]
	(No symbol) [0x00007FF66E3683A8]
	(No symbol) [0x00007FF66E369441]
	GetHandleVerifier [0x00007FF66E9225AD+4238317]
	GetHandleVerifier [0x00007FF66E95F70D+4488525]
	GetHandleVerifier [0x00007FF66E9579EF+4456495]
	GetHandleVerifier [0x00007FF66E600576+953270]
	(No symbol) [0x00007FF66E49E54F]
	(No symbol) [0x00007FF66E499224]
	(No symbol) [0x00007FF66E49935B]
	(No symbol) [0x00007FF66E489B94]
	BaseThreadInitThunk [0x00007FFFA28F257D+29]
	RtlUserThreadStart [0x00007FFFA36CAA48+40]


In [ ]:
# List to store the post image URLs
post_urls = []

for soup in soups:
    # Find all anchor elements with href attributes
    anchors = soup.find_all('a', href=True)
    
    # Filter URLs that start with "/p/" or "/reel/"
    post_urls.extend([anchor['href'] for anchor in anchors if anchor['href'].startswith(("/p/", "/reel/"))])

# Convert the list to a set to remove duplicates
unique_post_urls = list(set(post_urls))

print(f"before: {len(post_urls)}, after: {len(unique_post_urls)}")

before: 280, after: 120


In [ ]:
# json_list = []
# url_list = []
hashtag_list = []
# count2 = 0
# Define the query parameters to add
# query_parameters = "__a=1&__d=dis"

# go through all urls
for url in unique_post_urls:
    try:
        # Get the current URL of the page
        current_url = driver.current_url

        # Append the query parameters to the current URL
        modified_url = "https://www.instagram.com/" + url

        # Navigate to the post URL
        driver.get(modified_url)

        # Wait for the page to load (you might need to adjust the wait time)
        driver.implicitly_wait(15)

        caption_element = driver.find_element(By.XPATH, '/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div/span/div/span')
        caption_text = caption_element.text
        # Extract hashtags from elements
        # hashtags = [element.text for element in hashtag_elements]
        words = caption_text.split()

        # Extract words starting with #
        hashtags = [word for word in words if word.startswith('#')]
        hashtag_list.append((modified_url, hashtags))
        # if count2 == 20:
        #     break
        # count2+=1

    except (NoSuchElementException, TimeoutException, json.JSONDecodeError) as e:
        print(f"Error processing URL {url}: {e}")


Error processing URL /p/C5X3zuIPUnW/?next=%2F: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div/div/div[2]/div/div/div[1]/div[1]/div[2]/section/main/div/div[1]/div/div[2]/div/div[2]/div/div[1]/div/div[2]/div/span/div/span"}
  (Session info: chrome=123.0.6312.122); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF733927032+63090]
	(No symbol) [0x00007FF733892C82]
	(No symbol) [0x00007FF73372EC65]
	(No symbol) [0x00007FF73377499D]
	(No symbol) [0x00007FF733774ADC]
	(No symbol) [0x00007FF7337B5B37]
	(No symbol) [0x00007FF73379701F]
	(No symbol) [0x00007FF7337B3412]
	(No symbol) [0x00007FF733796D83]
	(No symbol) [0x00007FF7337683A8]
	(No symbol) [0x00007FF733769441]
	GetHandleVerifier [0x00007FF733D225AD+4238317]
	GetHandleVerifier [0x00007FF733D5F70D+4488525]
	GetHandleVerifier [0x00007FF733D579EF+44

KeyboardInterrupt: 

In [ ]:
# len(hashtag_list)

76

In [ ]:
import csv
# Open a CSV file in write mode
with open('instagram_data.csv', 'w', newline='', encoding='utf-8') as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)
    
    # Write header row
    csv_writer.writerow(['URL', 'Hashtags'])
    
    # Write data rows
    for post_url, hashtags in hashtag_list:
        csv_writer.writerow([post_url, ', '.join(hashtags)])